In [18]:
import Quandl
import pandas as pd
import numpy as np
import datetime

In [19]:
nasdaq_url = 'http://www.nasdaq.com/screening/companies-by-industry.aspx?exchange=NASDAQ&render=download'

In [20]:
df_tickers = pd.read_csv(nasdaq_url)

In [21]:
df_spy = Quandl.get('GOOG/NYSE_SPY', start_date='2010-01-01' , end_date="2005-12-31")

In [22]:
df_spy['ch'] =  df_spy['Close'].pct_change()

In [23]:
df_stock = Quandl.get('WIKI/AAPL', start_date='2010-01-01', end_date="2005-12-31")

In [26]:
df_stock = df_stock[df_stock.index > datetime.datetime(2010,1,1)]

In [27]:
df_stock['ch'] =  df_stock['Adj. Close'].pct_change()

In [10]:
# beta
risk_free = .005

In [29]:
df_spy  = df_spy[df_spy.index > datetime.datetime(2010,1,1)]

In [39]:
df_ch = df_stock[['ch']].join(df_spy['ch'], lsuffix='_stock').dropna()

In [42]:
df_ch

,ch_stock,ch
Date,,
2010-01-05,0.001729,0.002647
2010-01-06,-0.015906,0.000704
2010-01-07,-0.001849,0.004221
2010-01-08,0.006648,0.003328
2010-01-11,-0.008822,0.001397
2010-01-12,-0.011375,-0.009326
2010-01-13,0.014106,0.008446
2010-01-14,-0.005792,0.002705
2010-01-15,-0.016712,-0.011224


In [46]:
beta = np.cov(df_ch.T) / np.var(df_ch['ch'])

In [61]:
np.cov(df_ch['ch_stock'], df_ch['ch'])

array([[  2.71305724e-04,   8.95060286e-05],
       [  8.95060286e-05,   9.47142358e-05]])

In [59]:
np.cov(df_ch.T)

array([[  2.71305724e-04,   8.95060286e-05],
       [  8.95060286e-05,   9.47142358e-05]])

In [47]:
beta[0][1]

0.94554586728591816

In [54]:
idx = pd.date_range(start='01-01-2010', end='01-01-2016')
tickers = df_tickers['Symbol'].tolist()[:10]
dt = {t: np.random.uniform(-.05, .05, len(idx)) for t in tickers}

In [57]:
df = pd.DataFrame(data=dt, index=idx)

In [56]:
tickers

['PIH', 'FLWS', 'FCCY', 'SRCE', 'VNET', 'TWOU', 'JOBS', 'CAFD', 'EGHT', 'AVHI']

In [ ]:
def beta(stock, market):
    return np.cov(stock, market)[0][1] / np.var(stock)

In [63]:
df.rolling(window=200).apply(lambda x: beta(x['AVHI'], x['JOBS']))

Rolling [window=200,center=False,axis=0]

In [65]:
df.rolling(window=200).apply(beta)

,AVHI,CAFD,EGHT,FCCY,FLWS,JOBS,PIH,SRCE,TWOU,VNET
2010-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [66]:
# credit: http://stackoverflow.com/questions/37447347/dataframe-representation-of-a-rolling-window/37448165#37448165
